In [ ]:
import pandas as pd
import numpy as np

In [10]:
labels = pd.read_csv("/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/RAW_DATA/Code15/exams.csv")
labels.rename(columns={'exam_id': 'SID'}, inplace=True)
labels.rename(columns={'is_male': 'Is_Male'}, inplace=True)
labels.rename(columns={'age': 'Age'}, inplace=True)
labels.rename(columns={'death': 'Mort_Event'}, inplace=True)
labels.rename(columns={'timey': 'Mort_TTE'}, inplace=True)
labels.rename(columns={'patient_id': 'PID'}, inplace=True)

labels.loc[labels['Mort_TTE']<1/365,'Mort_TTE'] = 1/365


In [37]:
x

,index,SID,Age,Is_Male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,PID,Mort_Event,Mort_TTE,normal_ecg,trace_file
0,352,3158243,59,False,64.235565,False,False,False,False,False,False,1280570,NaN,NaN,False,exams_part0.hdf5
1,353,1126697,63,False,67.575520,False,False,False,False,False,False,223781,False,2.178080,True,exams_part0.hdf5
2,354,91288,49,False,55.587772,False,False,False,False,False,False,1546425,NaN,NaN,False,exams_part0.hdf5
3,355,2660251,42,False,48.175446,False,False,False,False,False,False,160919,False,7.243830,False,exams_part0.hdf5
4,356,4390974,42,True,47.617800,False,False,False,False,False,False,98649,NaN,NaN,False,exams_part0.hdf5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,344091,1133183,46,False,61.897950,False,False,False,False,False,False,161647,NaN,NaN,True,exams_part0.hdf5
19996,344092,531004,71,True,78.271890,False,False,False,False,False,False,110931,NaN,NaN,False,exams_part0.hdf5
19997,344093,1439610,37,False,37.154194,False,False,False,False,False,False,231567,False,1.545204,True,exams_part0.hdf5
19998,344094,3096455,74,True,76.501396,False,False,False,False,False,False,611873,False,6.589036,False,exams_part0.hdf5


In [38]:
import os
import h5py

EIDs = []
ECGs = []
Code15_ECG_Directory = "/mnt/disk1/aiotlab/dangnh/ecg-survival-benchmark/data/code15/ecg"

for f in os.listdir(Code15_ECG_Directory): 
    print(f)
    with h5py.File(os.path.join(Code15_ECG_Directory,f), "r") as h:
        print("Keys: %s" % h.keys())
        EIDs.append(h['exam_id'][:-1])
        ECGs.append(h['tracings'][:-1,648:-648,:].astype(np.float32))
        

ModuleNotFoundError: No module named 'h5py'

In [ ]:
Test_Train_split_12345 = []
for PID,TTE in zip(labels['PID'],labels['Mort_TTE']):
    if (np.isnan(TTE)):
        Test_Train_split_12345.append('NegTTE')
    else:
        Test_Train_split_12345.append(PID_labels[PID])

In [25]:
labels[np.isnan(labels["Mort_TTE"]) == False]

,SID,Age,Is_Male,nn_predicted_age,1dAVb,RBBB,LBBB,SB,ST,AF,PID,Mort_Event,Mort_TTE,normal_ecg,trace_file
0,1169160,38,True,40.160484,False,False,False,False,False,False,523632,False,2.098628,True,exams_part13.hdf5
1,2873686,73,True,67.059440,False,False,False,False,False,False,1724173,False,6.657529,False,exams_part13.hdf5
2,168405,67,True,79.621740,False,False,False,False,False,True,51421,False,4.282188,False,exams_part13.hdf5
3,271011,41,True,69.750260,False,False,False,False,False,False,1737282,False,4.038353,True,exams_part13.hdf5
4,384368,73,True,78.873460,False,False,False,False,False,False,331652,False,3.786298,False,exams_part13.hdf5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345774,1123951,33,True,35.893005,False,False,False,False,False,False,770553,False,2.189039,True,exams_part2.hdf5
345775,954704,73,False,68.169136,False,False,False,False,False,False,1044781,False,2.520546,False,exams_part2.hdf5
345776,589697,75,False,78.080810,False,False,False,False,False,False,1020589,False,3.304107,False,exams_part2.hdf5
345777,2780563,44,False,73.120636,False,False,False,False,False,False,178,False,7.339720,False,exams_part2.hdf5


In [28]:
np.isnan(labels['Mort_TTE'])==False

0         True
1         True
2         True
3         True
4         True
          ... 
345774    True
345775    True
345776    True
345777    True
345778    True
Name: Mort_TTE, Length: 345779, dtype: bool